In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

In [2]:
all_synthetic_trj = pd.read_csv('data_save/all_synthetic_trj.csv', header=0)
traj_info = pd.read_csv('data_save/all_state_change_point_record.csv', header=0)
results = pd.read_csv('data_save/scale_sensitivity_analysis_results.csv', header = 0)

In [3]:
def impacts_of_acc_plot(mean_mae, state_change_label, a_tar):
    folder_path = "figure_save_acc/"
    
    if os.path.isdir(folder_path) == False:
        os.makedirs(folder_path)
    
    plot_save_path = './figure_save_acc/' +'a_tar_'+str(a_tar)+'.png'
    
    plt.subplot(2,1,1)
    title = 'acc_'+str(a_tar)
    plt.title(title)
    for i in mean_mae.keys():
        plt.plot(range(2,128), mean_mae[i], label = state_change_label[i])
        plt.legend(fontsize=12)
        plt.ylabel('mae(s)',fontsize=16)
        plt.xlabel('scale',fontsize=16)
        plt.xlim(0, 64)
        plt.ylim(0, 2)
    #plt.show()

    plt.subplot(2,1,2)
    for i in mean_mae.keys():
        plt.plot(mean_mae[i], label = state_change_label[i])
        plt.legend(fontsize=12)
        plt.ylabel('mae(s)',fontsize=16)
        plt.xlabel('scale',fontsize=16)
        #plt.xlim(0, 50)
        #plt.ylim(0, 0.2)
    plt.savefig(plot_save_path, dpi=600)
    plt.close('all')

In [6]:
def impacts_of_acc(filter_condition, a_tar, results):
    
    state_change_label = {0: 'A-F', 1: 'A-D', 2: 'S-A', 3: 'F-D', 4: 'D-S'}
    mae_columns = [c for c in results.columns if 'mae' in c]
    
    all_mae = {}
    mean_mae = {}
    sub_result_dict = {}
     
    for look_at_label in [0, 2, 3, 4]:    

        sub_results = results[results['state_change_label']  == look_at_label]
        sub_result_dict[look_at_label] = sub_results

        if filter_condition:
            
            filter_results = sub_results
            #filter_results = sub_results[sub_results['test_jerk'] == jerk_tar]

            if look_at_label == 0:
                filter_results = filter_results[filter_results['test_acc'].astype('float') == a_tar]

            elif look_at_label == 2:
                filter_results = filter_results[filter_results['test_acc'].astype('float') == a_tar]

            elif look_at_label == 3:
                filter_results = filter_results[filter_results['test_dec'].astype('float') == -a_tar]

            elif look_at_label == 4:
                filter_results = filter_results[filter_results['test_dec'].astype('float') == -a_tar]

        else:
            filter_results = sub_results
        
        all_mae[look_at_label] = filter_results[mae_columns]

        mean_mae[look_at_label] = list(all_mae[look_at_label].mean(axis=0))
        
    #print('mean_mae\n', mean_mae)
    impacts_of_acc_plot(mean_mae, state_change_label, a_tar)

In [8]:
for a in [1.0, 1.5, 2, 2.5, 3.0]:
    impacts_of_acc(True, a, results)